In [150]:
import os, requests
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

In [151]:
fname = []
for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
    if not os.path.isfile(fname[j]):
        try:
            r = requests.get(url[j])
        except requests.ConnectionError:
            print("!!! Failed to download data !!!")
        else:
            if r.status_code != requests.codes.ok:
                print("!!! Failed to download data !!!")
            else:
                with open(fname[j], "wb") as fid:
                    fid.write(r.content)

In [152]:
alldat = np.array([])
for j in range(len(fname)):
    alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]

In [169]:
go = ~np.logical_and(dat['contrast_right'] == 0, dat['contrast_left'] == 0)

contrast_left = dat['contrast_left']
contrast_right = dat['contrast_right']

response =  np.abs(dat['response']) == 1 
contrast_diff_response = np.abs(contrast_left - contrast_right)*response.astype(int)*go.astype(int) 

one_trials = contrast_diff_response == 1
easy_trials = contrast_diff_response == 0.75
medium_trials = contrast_diff_response == 0.5
difficult_trials = contrast_diff_response == 0.25
zero_trials = contrast_diff_response == 0

print(len(contrast_diff_response))
print(sum(one_trials) + sum(easy_trials) + sum(zero_trials) + sum(medium_trials) + sum(difficult_trials))
print(sum(one_trials))
print(sum(easy_trials))
print(sum(medium_trials))
print(sum(zero_trials))
print(sum(difficult_trials))

340
340
43
44
65
140
48


In [214]:
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]

In [217]:
# select neurons in visual cortex and thalamus only 

keep_neurons = []
for ii,barea in enumerate(dat['brain_area']):
    if ((barea in brain_groups[0]) or (barea in brain_groups[1])):
        keep_neurons.append(ii)

In [220]:
spks = dat['spks']

In [222]:
spks = spks[keep_neurons,:,50:]

In [224]:
spks = spks.reshape((340, 300*200))

In [226]:
X = spks
y = contrast_diff_response

kf = KFold(n_splits=5)
kf.get_n_splits(X)
rmse_s = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    rmse = np.sqrt(((y_test-y_pred)**2).mean())
    rmse_s.append(rmse)
    print("RMSE: {}".format(rmse))
    


RMSE: 0.42099345747401345
RMSE: 0.3993181773892979
RMSE: 0.35154334169656126
RMSE: 0.3709750701356516
RMSE: 0.4014612452423425
